In [0]:
stage1_source_path = f"wasbs://dlbikestorestage1@adlsbikestoreinterns.blob.core.windows.net/mostafa"
stage2_source_path = f"wasbs://dlbikestorestage2@adlsbikestoreinterns.blob.core.windows.net/mostafa"
infomart_path = f"wasbs://dlbikestoreinfomart@adlsbikestoreinterns.blob.core.windows.net/mostafa/product_dim"


In [0]:
product_df = spark.read.format('delta').load(stage1_source_path + '/products')
brands_df = spark.read.format('delta').load(stage1_source_path + '/brands')
categories_df = spark.read.format('delta').load(stage1_source_path + '/categories')


In [0]:
product_df = product_df.drop("timestamp", "process_id")
product_dim = product_df.join(brands_df, product_df.brand_id == brands_df.brand_id, "left").drop('timestamp', 'process_id')
product_dim = product_dim.join(categories_df, product_dim.category_id == categories_df.category_id, "left")
product_dim = product_dim.drop('brand_id','category_id','source')
product_dim.drop('timestamp', 'process_id')
display(product_dim)

In [0]:
# Remove duplicate columns
product_dim = product_dim.toDF(
    *[f"{col}_{i}" if col == "time_stamp" else col for i, col in enumerate(product_dim.columns)]
)

# Write the DataFrame to Delta format
product_dim.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(infomart_path)